In [ ]:
%%capture
!pip install -q langgraph langchain langchain-community faiss-cpu pypdf langchain-core wikipedia arxiv openai sentence-transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
from typing import Dict, Any
from langgraph.graph import Graph, END
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_community.llms import HuggingFacePipeline

# **1. Initialize the LLM using HuggingFace pipeline (local execution, NO InferenceClient)**

In [ ]:
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=256,
    do_sample=False  # Deterministic output (temperature ignored when do_sample=False)
)
llm = HuggingFacePipeline(pipeline=pipe)

# **2. Define embedding model**

In [ ]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cpu'}
)

# **3. Document Retriever Class**

In [ ]:
class DocumentRetriever:
    def __init__(self, file_path: str):
        self.embedding_model = embedding_model
        self.vectorstore = None
        self.load_documents(file_path)

    def load_documents(self, file_path: str):
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Document not found at {file_path}")
        loader = PyPDFLoader(file_path)
        docs = loader.load()
        # Use a smaller chunk size to avoid long input prompts
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100)
        splits = text_splitter.split_documents(docs)
        self.vectorstore = FAISS.from_documents(splits, self.embedding_model)

    def retrieve(self, state: Dict[str, Any]) -> Dict[str, Any]:
        if not self.vectorstore:
            raise ValueError("Vectorstore not initialized")
        question = state.get("question", "")
        relevant_docs = self.vectorstore.similarity_search(question, k=3)
        return {
            "documents": relevant_docs,
            "question": question,
            "retry_count": state.get("retry_count", 0)
        }

# **4. Answer Generator Class**

In [ ]:
class AnswerGenerator:
    def __init__(self, llm):
        self.llm = llm

    def generate(self, state: Dict[str, Any]) -> Dict[str, Any]:
        context = "\n".join([doc.page_content for doc in state["documents"]])
        # Truncate context to avoid exceeding model's max input length (512 tokens for Flan-T5-Large)
        context = context[:1800]
        prompt = f"Based on the following context:\n{context}\n\nQuestion: {state['question']}\nAnswer:"
        try:
            result = self.llm(prompt)
            # HuggingFacePipeline returns a string or a list (depends on LangChain version)
            if isinstance(result, str):
                answer = result.strip()
            elif isinstance(result, list) and len(result) > 0 and "generated_text" in result[0]:
                answer = result[0]["generated_text"].strip()
            else:
                answer = str(result).strip()
            return {
                "answer": answer,
                "documents": state["documents"],
                "question": state["question"],
                "verified": False,
                "retry_count": state.get("retry_count", 0)
            }
        except Exception as e:
            print(f"Error generating answer: {str(e)}")
            return {
                "answer": "I couldn't generate an answer due to an error.",
                "documents": state["documents"],
                "question": state["question"],
                "verified": False,
                "retry_count": state.get("retry_count", 0)
            }

# **5. Answer Verifier Class**

In [ ]:
class AnswerVerifier:
    def __init__(self, llm):
        self.llm = llm

    def verify(self, state: Dict[str, Any]) -> Dict[str, Any]:
        context = "\n".join([doc.page_content for doc in state["documents"]])
        # Truncate context for verification as well
        context = context[:1800]
        verification_prompt = (
            f"Verify if this answer is correct based on the context:\n"
            f"Context: {context}\n"
            f"Question: {state['question']}\n"
            f"Answer: {state['answer']}\n"
            f"Respond only with 'True' or 'False':"
        )
        try:
            result = self.llm(verification_prompt)
            if isinstance(result, str):
                verification = result.strip().lower()
            elif isinstance(result, list) and len(result) > 0 and "generated_text" in result[0]:
                verification = result[0]["generated_text"].strip().lower()
            else:
                verification = str(result).strip().lower()
            is_verified = "true" in verification
            return {
                "verified": is_verified,
                "answer": state["answer"],
                "documents": state["documents"],
                "question": state["question"],
                "retry_count": state.get("retry_count", 0) + 1
            }
        except Exception as e:
            print(f"Error verifying answer: {str(e)}")
            return {
                "verified": False,
                "answer": state["answer"],
                "documents": state["documents"],
                "question": state["question"],
                "retry_count": state.get("retry_count", 0) + 1
            }

# **6. Workflow logic**

In [ ]:
def should_continue(state: Dict[str, Any]) -> str:
    if state["verified"]:
        return "end"
    if state.get("retry_count", 0) >= 2:
        return "end"
    return "retry"

workflow = Graph()

retriever = DocumentRetriever("computer_science_is_foundational.pdf")  # Update path if needed
generator = AnswerGenerator(llm)
verifier = AnswerVerifier(llm)

workflow.add_node("retriever", retriever.retrieve)
workflow.add_node("generator", generator.generate)
workflow.add_node("verifier", verifier.verify)

workflow.add_edge("retriever", "generator")
workflow.add_edge("generator", "verifier")

workflow.add_conditional_edges(
    "verifier",
    should_continue,
    {
        "end": END,
        "retry": "retriever"
    }
)

workflow.set_entry_point("retriever")
agent = workflow.compile()

In [ ]:
if __name__ == "__main__":
    try:
        print("Document QA System - Type 'exit' to quit")
        while True:
            question = input("\nEnter your question: ").strip()
            if question.lower() == 'exit':
                break
            if not question:
                print("Please enter a valid question.")
                continue

            result = agent.invoke({
                "question": question,
                "retry_count": 0
            })

            print("\nAnswer:", result["answer"])
            print("\nSource references:")
            for i, doc in enumerate(result["documents"], 1):
                print(f"{i}. {doc.page_content[:150]}...")
    except Exception as e:
        print(f"Error in main execution: {str(e)}")